<b> Run below cells from the folder that contains ads16_dataset/ unzipped </b>

In [296]:
import pandas as pd
import glob
import pathlib
import re

In [297]:
pd.set_option('display.max_colwidth', -1)

In [298]:
# Global constants
g_userPart1PathPrefix = "./ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Corpus/Corpus/"
g_userPart2PathPrefix = "./ads16-dataset/ADS16_Benchmark_part2/ADS16_Benchmark_part2/Corpus/Corpus/"
g_userIdPrefix = "U0"

g_adsPart1PathPrefix = "./ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/"
g_adsPart2PathPrefix = "./ads16-dataset/ADS16_Benchmark_part2/ADS16_Benchmark_part2/Ads/Ads/"

# UDFs

## UDFs for generating Users Dataset

In [299]:
def generate_data_User( pathPrefix, userId ):
    completePath = pathPrefix + userId + "/"
    
    # INF
    infFile = userId + "-INF.csv"
    userInf_df = pd.read_csv(completePath + infFile, delimiter=";")
    
    # Pref
    prefFile = userId + "-PREF.csv"
    userPref_df = pd.read_csv(completePath + prefFile, delimiter=";")
    
    user_df = pd.concat([userInf_df, userPref_df], axis=1)
    
    # Pos
    posFile = userId + "-IM-POS.csv"
    userPos_df = pd.read_csv(completePath + posFile, delimiter=";")
    userPos_df = userPos_df.iloc[1:]
    userPos_df.reset_index(drop=True, inplace=True)
    user_df = pd.concat([user_df, userPos_df], axis=1)

    # Neg
    negFile = userId + "-IM-NEG.csv"
    userNeg_df = pd.read_csv(completePath + negFile, delimiter=";")
    userNeg_df = userNeg_df.iloc[1:]
    userNeg_df.reset_index(drop=True, inplace=True)
    user_df = pd.concat([user_df, userNeg_df], axis=1)

    user_df.insert(0, "UserId", userId, True)
    # user_df = user_df.set_index('UserId')
    # user_df.info()
    
    return user_df

In [300]:
def generate_data_partUsers( usersPartPathPrefix, startRange, endRange ):
    partUsers_df = pd.DataFrame()
    
    for i in range(startRange, endRange):
        thisUserIdNum = str(i)
        thisUserId = g_userIdPrefix + thisUserIdNum.zfill(3)
        # print(thisUserId)
        thisUser_df = generate_data_User(usersPartPathPrefix, thisUserId)
        partUsers_df = partUsers_df.append(thisUser_df, sort=True)
        partUsers_df.set_index('UserId')
        
    return partUsers_df

In [301]:
def generate_data_allUsers():
    allUsers_df = pd.DataFrame()

    part1Users_df = generate_data_partUsers(g_userPart1PathPrefix, 1, 61)
    allUsers_df = allUsers_df.append(part1Users_df, sort=True)

    part2Users_df = generate_data_partUsers(g_userPart2PathPrefix, 61, 121)
    allUsers_df = allUsers_df.append(part2Users_df, sort=True)

    return allUsers_df

## UDFs for generating Ads Dataset

In [302]:
def generate_data_adCats():
    adCatsLst = [['01', "Clothing & Shoes", 16],
                 ['02', "Automotive", 15],
                 ['03', "Baby Products", 15],
                 ['04', "Health & Beauty", 15],
                 ['05', "Media (BMVD)", 15],
                 ['06', "Consumer Electronics", 15],
                 ['07', "Console & Video Games", 15],
                 ['08', "DIY & Tools", 15],
                 ['09', "Garden & Outdoor living", 15],
                 ['10', "Grocery", 15],
                 ['11', "Kitchen & Home", 15],
                 ['12', "Betting", 15],
                 ['13', "Jewellery & Watches", 15],
                 ['14', "Musical Instruments", 15],
                 ['15', "Office Products", 15],
                 ['16', "Pet Supplies", 15],
                 ['17', "Computer Software", 15],
                 ['18', "Sports & Outdoors", 15],
                 ['19', "Toys & Games", 15],
                 ['20', "Dating Sites", 15]
                ] 
    adCats_df = pd.DataFrame(adCatsLst, columns =['AdCatId', 'AdCatName', 'AdCatNumAds'])
    return adCats_df

In [303]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)',text.split('/')[-1].split('.')[0]) ]

def generate_data_partAds( adsPartPathPrefix, startRange, endRange ):
    partAds_df = pd.DataFrame()
    partAdsRows = []
    
    for i in range(startRange, endRange):
        iStr = str(i)
        adsFiles = pathlib.Path(adsPartPathPrefix + iStr + "/").glob("*.png")
        adsFileStrLst = []
        for adsFile in adsFiles:
            adsFileStr = str(adsFile)
            adsFileStrLst.append(adsFileStr)
        adsFileStrLst.sort(key=natural_keys)
    
        for adsFileStr in adsFileStrLst:
            adId = adsFileStr.split('/')[-1].split('.')[0]
            adId = "A" +  iStr.zfill(2) + "_" + adId.zfill(2)
        #    print(adId, adsFileStr)
            partAdsRows.append([adId, adsFileStr])
        
    partAds_df = pd.DataFrame(partAdsRows, columns =['AdId', 'AdFilePath'])
    partAds_df.set_index('AdId')
        
    return partAds_df


In [304]:
# DEBUG

def generate_data_allAds():
    allAds_df = pd.DataFrame()
    
    part1Ads_df = generate_data_partAds(g_adsPart1PathPrefix, 1, 11)
    allAds_df = allAds_df.append(part1Ads_df, sort=True)

    part2Ads_df = generate_data_partAds(g_adsPart2PathPrefix, 11, 21)
    allAds_df = allAds_df.append(part2Ads_df, sort=True)

    allAds_df = allAds_df.set_index('AdId')
    return allAds_df


## UDFs for generating Ratings Dataset

In [338]:
def df_crossjoin(df1, df2):
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey').drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res

# Generate datasets

## Generate Users dataset

In [305]:
allUsers_df = generate_data_allUsers()
allUsers_df = allUsers_df.set_index('UserId')

In [306]:
allUsers_df.head()

,Age,Cap/Zip-Code,Countries visited,Fave Sports,Gender,Home country,Home town,Income,Last Name,Most listened musics,...,fave6,fave7,fave8,fave9,unfave1,unfave2,unfave3,unfave4,unfave5,unfave6
UserId,,,,,,,,,,,,,,,,,,,,,
U0001,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",...,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN
U0002,26,60638,"Poland, Mexico, United States of America",Nothing,F,United States of America,Chicago,1,Hidden,"Dance Music, Electronic Music, Indie Pop, Pop (Popular music), Rock",...,NaN,NaN,NaN,NaN,seafood fish,pizza,bacon,bad odor,doglicks,NaN
U0003,22,54942,France,"Team sports (Footbal, Baseball, Rugby, ...)",M,United States of America,Greenville,1,Hidden,"Hip Hop - Rap, Dance Music, Electronic Music, Rock",...,NaN,NaN,NaN,NaN,Ew,Useless,Ice Cream,Play Dog,More Sweets,NaN
U0004,24,NW1 1EU,"United States of America, United Kingdom, Italy, Germany, France, Spain, Netherlands (Holland, Europe), Mexico, Aruba, Bermuda, Portugal","Individual sports‎ (Tennis, Archery, ...)",F,Great Britain,London,1,Hidden,"Alternative Music, Pop (Popular music), Rock",...,NaN,NaN,NaN,NaN,The word Moist,The word Crusty,Gross Toenails,Eye Crust,Nose Picking,NaN
U0005,34,10000,"Russia, Slovakia, Slovenia, China, India, France, Hungary, Italy, Great Britain, Ukraine","Individual sports‎ (Tennis, Archery, ...)",F,Czech Republic,Prague,1,Hidden,"Electronic Music, Asian Pop (J-Pop or K-pop), Rock",...,NaN,NaN,NaN,NaN,Fat people,Hairy bodies,Dirty hair,"Nicab, paranja",Junkies,NaN


In [307]:
allUsers_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, U0001 to U0120
Data columns (total 35 columns):
Age                           120 non-null int64
Cap/Zip-Code                  120 non-null object
Countries visited             120 non-null object
Fave Sports                   120 non-null object
Gender                        120 non-null object
Home country                  120 non-null object
Home town                     120 non-null object
Income                        120 non-null int64
Last Name                     120 non-null object
Most listened musics          120 non-null object
Most read books               120 non-null object
Most visited websites         120 non-null object
Most watched movies           120 non-null object
Most watched tv programmes    120 non-null object
Name                          120 non-null object
Paypal                        120 non-null object
Timepass                      120 non-null object
Type of Job                   120 non-null obj

In [308]:
allUsers_df.to_csv("AllUsers.csv", index=True)

## Generate Ads Categories Dataset

In [309]:
adCats_df = generate_data_adCats()
adCats_df = adCats_df.set_index('AdCatId')

In [310]:
adCats_df.head()

,AdCatName,AdCatNumAds
AdCatId,,
01,Clothing & Shoes,16
02,Automotive,15
03,Baby Products,15
04,Health & Beauty,15
05,Media (BMVD),15


In [311]:
adCats_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 01 to 20
Data columns (total 2 columns):
AdCatName      20 non-null object
AdCatNumAds    20 non-null int64
dtypes: int64(1), object(1)
memory usage: 480.0+ bytes


In [312]:
adCats_df.to_csv("AdCats.csv", index=True)

## Generate Ads Dataset

In [313]:
allAds_df = generate_data_allAds()

In [314]:
allAds_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 301 entries, A01_01 to A20_15
Data columns (total 1 columns):
AdFilePath    301 non-null object
dtypes: object(1)
memory usage: 4.7+ KB


In [315]:
allAds_df.head()

,AdFilePath
AdId,
A01_01,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/1.png
A01_02,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/2.png
A01_03,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/3.png
A01_04,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/4.png
A01_05,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/5.png


In [316]:
allAds_df.to_csv("AllAds.csv", index=True)

## Generate Users\*Ads Dataset

In [378]:
allUsers_And_Ads_df = df_crossjoin(allUsers_df, allAds_df)

In [379]:
allUsers_And_Ads_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 36120 entries, (U0001, A01_01) to (U0120, A20_15)
Data columns (total 36 columns):
Age                           36120 non-null int64
Cap/Zip-Code                  36120 non-null object
Countries visited             36120 non-null object
Fave Sports                   36120 non-null object
Gender                        36120 non-null object
Home country                  36120 non-null object
Home town                     36120 non-null object
Income                        36120 non-null int64
Last Name                     36120 non-null object
Most listened musics          36120 non-null object
Most read books               36120 non-null object
Most visited websites         36120 non-null object
Most watched movies           36120 non-null object
Most watched tv programmes    36120 non-null object
Name                          36120 non-null object
Paypal                        36120 non-null object
Timepass                      36120 n

In [380]:
allUsers_And_Ads_df = allUsers_And_Ads_df.reset_index()
allUsers_And_Ads_df.rename(columns={'level_0':'UserId'}, inplace=True)
allUsers_And_Ads_df.rename(columns={'level_1':'AdId'}, inplace=True)
allUsers_And_Ads_df.head(302)

,UserId,AdId,Age,Cap/Zip-Code,Countries visited,Fave Sports,Gender,Home country,Home town,Income,Last Name,Most listened musics,Most read books,Most visited websites,Most watched movies,Most watched tv programmes,Name,Paypal,Timepass,Type of Job,Weekly working hours,fave1,fave10,fave2,fave3,fave4,fave5,fave6,fave7,fave8,fave9,unfave1,unfave2,unfave3,unfave4,unfave5,unfave6,AdFilePath
0,U0001,A01_01,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/1.png
1,U0001,A01_02,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/2.png
2,U0001,A01_03,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/3.png
3,U0001,A01_04,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/4.png
4,U0001,A01_05,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/5.png
5,U0001,A01_06,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,Hidden,"Classical Music, Easy Listening, Jazz",Mystery,"Media (Books, DVD, CD/DVD Music) sites, Grocery &amp; beverages sites, Pet supplies sites","Action, Thriller, Drama, Comedy, Mystery","Comedy, Drama",Hidden,hidden@comcast.net,Internet,Housewife/Househusband,Full Time,my cats,NaN,my cats,movie we are in,tv show we are in,movie we are in,NaN,NaN,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,ads16-dataset/ADS16_Benchmark_part1/ADS16_Benchmark_part1/Ads/Ads/1/6.png
6,U0001,A01_07,62,15613,U

In [382]:
allUsers_And_Ads_df.to_csv("AllUsers_And_Ads.csv", index=False)

## Generate UsersRatings Dataset

In [390]:
# TODO: Move this UDF to top UDFs section
def generate_data_RatingsPerUser( pathPrefix, userId ):
    completePath = pathPrefix + userId + "/"

    data = ""
    rtFile = userId + "-RT.csv"
    rtNewFile = userId + "-RT-NEW.csv"
    
    with open(completePath + rtFile) as file:
        data = file.read().replace("\"", "")

    with open(completePath + rtNewFile,"w") as file:
        file.write(data)

    my_cols = [str(i) for i in range(300)]
    data3 = pd.read_csv(completePath + rtNewFile, sep=";|,", names=my_cols, header=None, engine="python")
    data3 = data3.iloc[2:]
    data3.reset_index(drop = True, inplace = True)
    
    for i in range(20):
        index = str(i)
        data3[index] = data3[index].astype('float64')
    
    data3 = data3.transpose()
    
    data3.insert(0, "UserId", userId, True)
    
    return data3

In [391]:
# TODO: Add a UDF that iterates over all Users
data4 = generate_data_RatingsPerUser(g_userPart1PathPrefix, "U0001")

In [392]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 299
Data columns (total 2 columns):
UserId    300 non-null object
0         300 non-null float64
dtypes: float64(1), object(1)
memory usage: 17.0+ KB


In [393]:
data4.head()

,UserId,0
0,U0001,1.0
1,U0001,1.0
2,U0001,1.0
3,U0001,1.0
4,U0001,1.0


# Scratchpad